## 03 – Integration & Automation

🟩 **GOOD:** This notebook demonstrates production-grade integration and automation patterns for CodeCraft AI, following Clean Architecture and AWS-native best practices.

### Purpose
- Show how to automate AI workflows and integrate with AWS-native services (e.g., S3, Lambda, Step Functions)
- Provide reproducible, testable integration experiments for stakeholders and technical reviewers
- Serve as a reference for CI/CD, event-driven ML, and operational automation

### Prerequisites
- The backend API service must be running and accessible at the endpoint specified by `CODECRAFT_API_URL`.
- All secrets and config must be injected via environment variables (never hardcoded).
- This notebook is a client only; all business logic and data processing are handled by the backend or AWS services.

> 🟦 **NOTE:** For local dev, start your FastAPI server with:
> `poetry run uvicorn src.adapters.api.main:app --reload`

### Environment-Aware Configuration

🟦 **NOTE:** All configuration and secrets are injected via environment variables for security and portability. No values are hardcoded.

In [2]:
import os
import requests

API_URL = os.getenv("CODECRAFT_API_URL", "http://localhost:8000/query")
API_AUTH_TOKEN = os.getenv("CODECRAFT_API_TOKEN", "")
API_AUTH_HEADER = os.getenv("CODECRAFT_API_AUTH_HEADER", "Authorization")
API_AUTH_PREFIX = os.getenv("CODECRAFT_API_AUTH_PREFIX", "Bearer")

def ask_ai(query: str, top_k: int = 3, extra_payload: dict = None):
    payload = {"query": query, "top_k": top_k}
    if extra_payload:
        payload.update(extra_payload)
    headers = {}
    if API_AUTH_TOKEN:
        if API_AUTH_PREFIX:
            headers[API_AUTH_HEADER] = f"{API_AUTH_PREFIX} {API_AUTH_TOKEN}"
        else:
            headers[API_AUTH_HEADER] = API_AUTH_TOKEN
    try:
        response = requests.post(API_URL, json=payload, headers=headers, timeout=30)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"🟥 CRITICAL: API error: {e}")
        return None

### Example: Batch Inference Automation

🟩 **GOOD:** Demonstrates how to automate batch queries and store results in S3 (or locally for dev).

In [3]:
import json
from datetime import datetime

batch_prompts = [
    "Summarize the main features of CodeCraft AI.",
    "How does CodeCraft AI handle data privacy?",
    "Describe how CodeCraft AI supports cost optimization in AWS."
]

batch_results = []
for prompt in batch_prompts:
    result = ask_ai(prompt)
    batch_results.append({"prompt": prompt, "result": result})

# 🟦 NOTE: Save results locally; in production, upload to S3 using boto3
output_path = f"batch_inference_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(batch_results, f, ensure_ascii=False, indent=2)
print(f"Batch inference results saved to {output_path}")

Batch inference results saved to batch_inference_20250625_172953.json


### Example: Event-Driven Automation (AWS Lambda Pattern)

🟦 **NOTE:** This cell shows how you might structure a Lambda handler for event-driven AI inference. For real AWS deployment, move this logic to a Lambda function and inject config via environment variables.

In [4]:
def lambda_handler(event, context=None):
    """
    Example Lambda handler for AI inference automation.
    Expects event['prompt'] and optional event['top_k'].
    """
    prompt = event.get("prompt")
    top_k = event.get("top_k", 3)
    if not prompt:
        return {"statusCode": 400, "body": "Missing 'prompt' in event."}
    result = ask_ai(prompt, top_k=top_k)
    return {"statusCode": 200, "body": result}

# 🟦 NOTE: Example local invocation
if __name__ == "__main__":
    test_event = {"prompt": "Summarize the main features of CodeCraft AI."}
    print(lambda_handler(test_event))

{'statusCode': 200, 'body': {'results': []}}
